In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 'Import Libraries'

import google.generativeai as genai
import requests
import json
import re
from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown, display, JSON, HTML
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import base64
from datetime import datetime
import ipywidgets as widgets
from IPython.display import clear_output
import sys

In [3]:
# 'Configure Gemini API'


# Configure Gemini API - Get from Kaggle secrets
try:
    api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash')
    print("✓ Gemini API configured successfully")
except Exception as e:
    print(f"✗ Error configuring Gemini API: {e}")
    print("Make sure you've added GEMINI_API_KEY to your Kaggle secrets")

✓ Gemini API configured successfully


In [4]:
#'Initialize Global Variables'


# Global cache for nutrition data
nutrition_cache = {}

# Load fallback nutrition data for common ingredients
fallback_data = {
    "beef": {"calories": 250, "protein": "25.0g", "fat": "15.0g", "carbs": "0.0g"},
    "chicken": {"calories": 165, "protein": "31.0g", "fat": "3.6g", "carbs": "0.0g"},
    "potato": {"calories": 160, "protein": "4.0g", "fat": "0.2g", "carbs": "37.0g"},
    "carrot": {"calories": 50, "protein": "1.2g", "fat": "0.3g", "carbs": "12.0g"},
    "onion": {"calories": 45, "protein": "1.1g", "fat": "0.1g", "carbs": "10.0g"},
    "olive oil": {"calories": 120, "protein": "0.0g", "fat": "14.0g", "carbs": "0.0g"},
    "broth": {"calories": 10, "protein": "1.0g", "fat": "0.5g", "carbs": "1.0g"},
    "flour": {"calories": 100, "protein": "3.0g", "fat": "0.3g", "carbs": "21.0g"},
    "rice": {"calories": 130, "protein": "2.7g", "fat": "0.3g", "carbs": "28.0g"},
    "pasta": {"calories": 190, "protein": "7.0g", "fat": "1.0g", "carbs": "37.0g"},
    "milk": {"calories": 60, "protein": "3.2g", "fat": "3.2g", "carbs": "4.8g"},
    "egg": {"calories": 70, "protein": "6.0g", "fat": "5.0g", "carbs": "0.6g"},
    "butter": {"calories": 100, "protein": "0.1g", "fat": "11.5g", "carbs": "0.0g"},
    "sugar": {"calories": 50, "protein": "0.0g", "fat": "0.0g", "carbs": "12.5g"},
    "bread": {"calories": 80, "protein": "3.0g", "fat": "1.0g", "carbs": "15.0g"},
    # Add more common ingredients
    "broccoli": {"calories": 55, "protein": "3.7g", "fat": "0.6g", "carbs": "11.2g", "fiber": "5.1g"},
    "soy sauce": {"calories": 8, "protein": "0.8g", "fat": "0.0g", "carbs": "0.8g", "sodium": "902.0mg"},
    "honey": {"calories": 64, "protein": "0.1g", "fat": "0.0g", "carbs": "17.3g", "sugar": "17.2g"},
    "garlic": {"calories": 4, "protein": "0.2g", "fat": "0.0g", "carbs": "1.0g"},
    "ginger": {"calories": 6, "protein": "0.2g", "fat": "0.1g", "carbs": "1.3g"},
    "red pepper flakes": {"calories": 6, "protein": "0.2g", "fat": "0.3g", "carbs": "1.1g"},
    "sesame seeds": {"calories": 52, "protein": "1.6g", "fat": "4.5g", "carbs": "2.1g", "fiber": "1.1g"}
}

In [5]:
#' Ingredient Processing Functions'

def extract_main_ingredient(ingredient_string):
    """Extract the main ingredient for better API matching"""
    # Define patterns for removing quantities, units, and preparations
    quantity_pattern = r'^\d+(/\d+)?(\.\d+)?\s*(lb|lbs|pound|pounds|oz|ounce|ounces|g|gram|grams|kg|cup|cups|tbsp|tablespoon|tablespoons|tsp|teaspoon|teaspoons)\s+'
    prep_pattern = r'(\s*,.*|\s*\(.*\)|\s+peeled|\s+chopped|\s+diced|\s+sliced|\s+cubed|\s+cut.*|\s+to taste|\s+for.*)'
    size_pattern = r'\s+(small|medium|large)\s+'
    
    # Apply patterns
    cleaned = re.sub(quantity_pattern, '', ingredient_string, flags=re.IGNORECASE)
    cleaned = re.sub(prep_pattern, '', cleaned, flags=re.IGNORECASE)
    cleaned = re.sub(size_pattern, ' ', cleaned, flags=re.IGNORECASE)
    
    # Get main ingredient
    main_ingredient = cleaned.strip().lower()
    
    # Fallback if nothing remains
    if not main_ingredient or len(main_ingredient) < 2:
        # Try to get first word after numbers and units
        words = ingredient_string.split()
        for word in words:
            if not (word.lower() in ['cup', 'cups', 'tbsp', 'tsp', 'pound', 'lb', 'g', 'kg', 'oz'] or 
                    re.match(r'^\d+(/\d+)?(\.\d+)?$', word)):
                return word.lower()
        return words[-1].lower() if words else "ingredient"
    
    return main_ingredient

In [6]:
#'Nutrition Data Retrieval Functions'

def get_nutrition_data(ingredients):
    """Get nutrition data for ingredients from Edamam API"""
    results = []
    api_url = "https://api.edamam.com/api/nutrition-data"
    
    # Try to get API keys from Kaggle secrets
    try:
        edamam_app_id = UserSecretsClient().get_secret("EDAMAM_APP_ID")
        edamam_app_key = UserSecretsClient().get_secret("EDAMAM_APP_KEY")
        print("✓ Edamam API keys loaded")
    except Exception as e:
        print(f"✗ Error loading Edamam API keys: {e}")
        print("Using fallback data only")
        edamam_app_id = None
        edamam_app_key = None
    
    for ingredient in ingredients:
        try:
            # Extract main ingredient for API query
            main_ingredient = extract_main_ingredient(ingredient)
            print(f"Extracted '{main_ingredient}' from '{ingredient}'")
            
            # Check cache first
            if main_ingredient in nutrition_cache:
                print(f"Using cached data for '{main_ingredient}'")
                results.append({
                    "ingredient": ingredient,
                    "main_ingredient": main_ingredient,
                    "nutrition_data": nutrition_cache[main_ingredient].copy()
                })
                continue
            
            # Check for matches in fallback data
            nutrient_data = None
            for key, data in fallback_data.items():
                if key in main_ingredient:
                    nutrient_data = data.copy()
                    print(f"Using fallback data for '{main_ingredient}'")
                    break
            
            # If no fallback match and API keys available, query the API
            if not nutrient_data and edamam_app_id and edamam_app_key:
                params = {
                    "app_id": edamam_app_id,
                    "app_key": edamam_app_key,
                    "ingr": main_ingredient
                }
                
                print(f"Querying API for '{main_ingredient}'")
                response = requests.get(api_url, params=params, timeout=10)
                response.raise_for_status()
                data = response.json()
                
                # Extract nutrition data
                nutrient_data = {"calories": data.get("calories", 0)}
                
                nutrient_map = {
                    "PROCNT": "protein",
                    "FAT": "fat", 
                    "CHOCDF": "carbs",
                    "FIBTG": "fiber",
                    "SUGAR": "sugar",
                    "NA": "sodium"
                }
                
                for code, name in nutrient_map.items():
                    if code in data.get("totalNutrients", {}):
                        nutrient_data[name] = f"{data['totalNutrients'][code]['quantity']:.1f}{data['totalNutrients'][code]['unit']}"
                    else:
                        nutrient_data[name] = "0.0g"
            
            # If still no data, use generic values
            if not nutrient_data:
                nutrient_data = {"calories": 50, "protein": "2.0g", "fat": "1.0g", "carbs": "5.0g"}
                print(f"Using generic values for '{main_ingredient}'")
            
            # Add to cache
            nutrition_cache[main_ingredient] = nutrient_data
            
            results.append({
                "ingredient": ingredient,
                "main_ingredient": main_ingredient,
                "nutrition_data": nutrient_data
            })
            
        except Exception as e:
            print(f"Error processing '{ingredient}': {e}")
            results.append({
                "ingredient": ingredient,
                "error": f"API Error: {str(e)}"
            })
    
    return results

In [7]:
#'Nutrition Calculation Functions'

def calculate_total_nutrition(nutrition_items):
    """Calculate total nutrition from all ingredients"""
    total = {"calories": 0, "protein": 0, "fat": 0, "carbs": 0, "fiber": 0, "sugar": 0, "sodium": 0}
    
    for item in nutrition_items:
        if 'nutrition_data' in item:
            # Add calories
            total["calories"] += item["nutrition_data"].get("calories", 0)
            
            # Add other nutrients
            for nutrient in total.keys():
                if nutrient != "calories" and nutrient in item["nutrition_data"]:
                    value_str = item["nutrition_data"][nutrient]
                    try:
                        value = float(re.search(r'([\d.]+)', value_str).group(1))
                        total[nutrient] += value
                    except:
                        pass
    
    # Format nutrition values
    for nutrient in total.keys():
        if nutrient != "calories":
            total[nutrient] = f"{total[nutrient]:.1f}g"
    
    return total

def calculate_per_serving(total_nutrition, servings=2):
    """Calculate nutrition per serving"""
    per_serving = {"calories": round(total_nutrition["calories"] / servings)}
    
    for nutrient in total_nutrition:
        if nutrient != "calories":
            match = re.search(r'([\d.]+)', total_nutrition[nutrient])
            if match:
                value = float(match.group(1)) / servings
                per_serving[nutrient] = f"{value:.1f}g"
    
    return per_serving

In [8]:
#'Recipe Generation Function'

def generate_recipe(user_ingredients=None, servings=2, dietary_preferences=None):
    """Generate a recipe using Gemini AI with few-shot prompting"""
    
    # Create prompt based on user ingredients or use default
    if user_ingredients and len(user_ingredients) > 0:
        ingredient_list = ", ".join(user_ingredients)
    else:
        ingredient_list = "beef, potatoes, and carrots"
    
    # Build dietary preferences string
    preferences_str = ""
    if dietary_preferences:
        pref_list = []
        for key, value in dietary_preferences.items():
            if value:
                pref_list.append(key)
        
        if pref_list:
            preferences_str = f" The recipe must be {', '.join(pref_list)}."
    
    # Create few-shot examples
    few_shot_examples = """
Example 1:
Input: Create a 4-serving recipe using chicken, tomatoes, and pasta. The recipe must be lowSodium.
Output:
{
  "recipe_name": "Fresh Herb and Lemon Chicken Pasta",
  "ingredients": [
    "1 lb boneless skinless chicken breast, cut into strips",
    "3 cups cherry tomatoes, halved",
    "12 oz whole grain pasta",
    "3 cloves garlic, minced",
    "1/4 cup fresh basil, chopped",
    "2 tbsp olive oil",
    "1 lemon, zested and juiced",
    "1/2 tsp black pepper",
    "1/4 cup grated Parmesan cheese (optional)"
  ],
  "instructions": [
    "Bring a large pot of unsalted water to boil and cook pasta according to package directions.",
    "Meanwhile, heat olive oil in a large skillet over medium heat.",
    "Add chicken strips and cook for 5-6 minutes until nearly done.",
    "Add garlic and cook for 30 seconds until fragrant.",
    "Add tomatoes and cook for 2-3 minutes until they start to soften.",
    "Drain pasta and add to the skillet with chicken and tomatoes.",
    "Add lemon zest, lemon juice, and black pepper. Toss to combine.",
    "Remove from heat and stir in fresh basil.",
    "Serve with a sprinkle of Parmesan if desired."
  ],
  "cooking_time": "25 minutes",
  "cuisine_type": "Italian-American",
  "difficulty": "easy",
  "prep_time": "15 minutes",
  "total_time": "40 minutes",
  "meal_type": "dinner",
  "tags": ["low-sodium", "high-protein", "dairy-optional"]
}

Example 2:
Input: Create a 2-serving recipe using tofu, bell peppers, and broccoli. The recipe must be vegan, glutenFree.
Output:
{
  "recipe_name": "Stir-Fried Tofu with Colorful Vegetables",
  "ingredients": [
    "1 block (14 oz) extra-firm tofu, pressed and cubed",
    "1 red bell pepper, sliced",
    "1 yellow bell pepper, sliced",
    "2 cups broccoli florets",
    "2 cloves garlic, minced",
    "1 tbsp ginger, grated",
    "3 tbsp gluten-free tamari sauce",
    "1 tbsp maple syrup",
    "1 tbsp rice vinegar",
    "2 tbsp vegetable oil",
    "1 tsp cornstarch mixed with 2 tbsp water",
    "2 green onions, sliced",
    "1 tbsp sesame seeds"
  ],
  "instructions": [
    "Press tofu for 30 minutes to remove excess water, then cut into 1-inch cubes.",
    "Mix tamari sauce, maple syrup, and rice vinegar in a small bowl.",
    "Heat oil in a large wok or skillet over medium-high heat.",
    "Add tofu cubes and cook until golden brown on all sides, about 5-7 minutes. Remove from pan and set aside.",
    "In the same pan, add garlic and ginger, stir for 30 seconds.",
    "Add bell peppers and broccoli, stir-fry for 3-4 minutes until crisp-tender.",
    "Return tofu to the pan, add sauce mixture and bring to a simmer.",
    "Add cornstarch slurry and cook until sauce thickens, about 1 minute.",
    "Garnish with green onions and sesame seeds before serving."
  ],
  "cooking_time": "15 minutes",
  "cuisine_type": "Asian-Fusion",
  "difficulty": "medium",
  "prep_time": "40 minutes",
  "total_time": "55 minutes",
  "meal_type": "dinner",
  "tags": ["vegan", "gluten-free", "high-protein", "stir-fry"]
}

Example 3:
Input: Create a 6-serving recipe using ground beef, beans, and tomatoes. The recipe must be lowCarb.
Output:
{
  "recipe_name": "Hearty Low-Carb Beef Chili",
  "ingredients": [
    "2 lbs lean ground beef",
    "1 large onion, diced",
    "3 cloves garlic, minced",
    "2 cans (14.5 oz each) diced tomatoes",
    "1 can (6 oz) tomato paste",
    "1 can (4 oz) diced green chilies",
    "1 can (15 oz) black soybeans, drained and rinsed",
    "2 tbsp chili powder",
    "1 tbsp cumin",
    "1 tsp dried oregano",
    "1/2 tsp cayenne pepper (optional)",
    "2 cups beef broth",
    "2 bell peppers, diced",
    "Salt and pepper to taste",
    "Optional toppings: shredded cheese, sour cream, diced avocado, chopped cilantro"
  ],
  "instructions": [
    "In a large pot or Dutch oven, brown the ground beef over medium-high heat, breaking it up as it cooks.",
    "Add the diced onion to the pot and cook until translucent, about 5 minutes.",
    "Add the garlic and cook for 30 seconds until fragrant.",
    "Stir in the diced tomatoes, tomato paste, green chilies, and black soybeans.",
    "Add the chili powder, cumin, oregano, cayenne (if using), and beef broth.",
    "Bring to a simmer, then reduce heat to low and cover.",
    "Simmer for 30 minutes, stirring occasionally.",
    "Add the diced bell peppers and simmer uncovered for another 15 minutes.",
    "Season with salt and pepper to taste.",
    "Serve with optional toppings as desired."
  ],
  "cooking_time": "45 minutes",
  "cuisine_type": "American",
  "difficulty": "easy",
  "prep_time": "15 minutes",
  "total_time": "60 minutes",
  "meal_type": "dinner",
  "tags": ["low-carb", "high-protein", "bean-free", "dairy-free"]
}
"""
    
    # Create the actual prompt
    prompt = f"""I'll show you examples of recipe generation tasks followed by their outputs. Then I'll give you a new task.

{few_shot_examples}

Now, create a {servings}-serving recipe using {ingredient_list}.{preferences_str}
Output in strict JSON format with:
- recipe_name (string)
- ingredients (list of strings with quantities)
- instructions (list of steps)
- cooking_time (string)
- cuisine_type (string - e.g. Italian, Mexican, etc.)
- difficulty (string - easy, medium, or hard)
- prep_time (string)
- total_time (string)
- meal_type (string - breakfast, lunch, dinner, dessert, snack)
- tags (list of strings - dietary tags like "vegetarian", "low-carb", etc.)
"""
    
    try:
        # Generate recipe from Gemini
        print("Generating recipe using Gemini AI with few-shot prompting...")
        response = model.generate_content(
            prompt,
            generation_config={"response_mime_type": "application/json"}
        )

        # Parse and clean up response
        recipe_data = json.loads(response.text)
        if isinstance(recipe_data, list):
            recipe_data = recipe_data[0]
        
        print("✓ Recipe generated successfully")
        return recipe_data
    except Exception as e:
        print(f"✗ Error generating recipe: {e}")
        return None

In [9]:
# 'Display Functions'

def create_nutrition_chart(per_serving):
    """Create a pie chart of macronutrients"""
    try:
        # Extract macronutrient values
        protein = float(re.search(r'([\d.]+)', per_serving["protein"]).group(1))
        fat = float(re.search(r'([\d.]+)', per_serving["fat"]).group(1))
        carbs = float(re.search(r'([\d.]+)', per_serving["carbs"]).group(1))
        
        # Calculate calorie contribution (rough estimate)
        protein_cals = protein * 4
        fat_cals = fat * 9
        carbs_cals = carbs * 4
        total_cals = protein_cals + fat_cals + carbs_cals
        
        # Adjust in case total doesn't match
        if total_cals > 0:
            protein_pct = (protein_cals / total_cals) * 100
            fat_pct = (fat_cals / total_cals) * 100
            carbs_pct = (carbs_cals / total_cals) * 100
        else:
            protein_pct = fat_pct = carbs_pct = 0
            
        # Create chart
        plt.figure(figsize=(6, 4))
        labels = ['Protein', 'Fat', 'Carbs']
        sizes = [protein_pct, fat_pct, carbs_pct]
        colors = ['#ff9999', '#66b3ff', '#99ff99']
        
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        plt.axis('equal')
        plt.title('Macronutrient Distribution')
        
        # Display chart directly
        plt.show()
        
    except Exception as e:
        print(f"✗ Error creating nutrition chart: {e}")

def display_recipe(recipe_data):
    """Display formatted recipe with nutrition information"""
    recipe = recipe_data["recipe"]
    
    # Recipe display
    recipe_md = f"""
# {recipe.get('recipe_name', 'Recipe')}

**Cuisine:** {recipe.get('cuisine_type', 'N/A')}  
**Difficulty:** {recipe.get('difficulty', 'N/A')}  
**Total Time:** {recipe.get('total_time', recipe.get('cooking_time', 'N/A'))}  
**Servings:** {recipe_data.get('servings', 2)}  
**Meal Type:** {recipe.get('meal_type', 'N/A')}  

## Ingredients
{chr(10).join('- ' + ing for ing in recipe['ingredients'])}

## Instructions
{chr(10).join(f"{i+1}. {step}" for i, step in enumerate(recipe['instructions']))}
"""

    # Display recipe
    display(Markdown(recipe_md))
    
    # Display nutrition information
    nutrition_md = """
## Nutrition Facts (per serving)
"""
    per_serving = recipe_data["nutrition"]["per_serving"]
    nutrition_md += f"- **Calories:** {per_serving['calories']}\n"
    for nutrient in ["protein", "fat", "carbs", "fiber", "sugar", "sodium"]:
        if nutrient in per_serving:
            nutrition_md += f"- **{nutrient.capitalize()}:** {per_serving[nutrient]}\n"
    
    display(Markdown(nutrition_md))
    
    # Display chart
    create_nutrition_chart(per_serving)
    
    # Display JSON output
    print("\n## JSON Output\n")
    # Format the JSON with indentation for readability
    formatted_json = json.dumps(recipe_data, indent=2)
    print(formatted_json)
    
    # Create a collapsible JSON output using IPython widgets
    display(HTML("""
    <details>
        <summary><b>View Recipe JSON</b></summary>
        <pre>{}</pre>
    </details>
    """.format(formatted_json.replace("<", "&lt;").replace(">", "&gt;"))))

In [10]:
# 'Main Recipe Generation Function'

def get_recipe_with_nutrition(user_ingredients=None, servings=2, dietary_preferences=None):
    """Main function to get a recipe with nutrition information"""
    
    # Show progress
    print("Starting recipe generation process...")
    
    # Generate recipe
    recipe = generate_recipe(user_ingredients, servings, dietary_preferences)
    if not recipe:
        print("⚠️ Failed to generate recipe")
        return None
    
    # Get nutrition data for ingredients
    print("Analyzing nutrition data...")
    nutrition_data = get_nutrition_data(recipe["ingredients"])
    
    # Calculate totals
    total_nutrition = calculate_total_nutrition(nutrition_data)
    per_serving = calculate_per_serving(total_nutrition, servings)
    
    # Create complete recipe data
    complete_recipe = {
        "recipe": recipe,
        "nutrition": {
            "ingredients": nutrition_data,
            "total": total_nutrition,
            "per_serving": per_serving
        },
        "servings": servings,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Display results
    print("✓ Recipe complete! Displaying results:")
    display_recipe(complete_recipe)
    
    return complete_recipe

In [11]:
#'Interactive Recipe App Function'

def create_interactive_recipe_app():
    # Create widgets
    ingredients_input = widgets.Text(
        value='',
        placeholder='Enter ingredients separated by commas (e.g. chicken, rice, broccoli)',
        description='Ingredients:',
        disabled=False,
        layout=widgets.Layout(width='80%')
    )
    
    servings_slider = widgets.IntSlider(
        value=2,
        min=1,
        max=10,
        step=1,
        description='Servings:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    
    vegetarian_checkbox = widgets.Checkbox(
        value=False,
        description='Vegetarian',
        disabled=False
    )
    
    vegan_checkbox = widgets.Checkbox(
        value=False,
        description='Vegan',
        disabled=False
    )
    
    gluten_free_checkbox = widgets.Checkbox(
        value=False,
        description='Gluten Free',
        disabled=False
    )
    
    low_carb_checkbox = widgets.Checkbox(
        value=False,
        description='Low Carb',
        disabled=False
    )
    
    low_sodium_checkbox = widgets.Checkbox(
        value=False,
        description='Low Sodium',
        disabled=False
    )
    
    # Option to save recipe as JSON file
    save_json_checkbox = widgets.Checkbox(
        value=False,
        description='Save Recipe as JSON',
        disabled=False
    )
    
    # Create a button to generate recipe
    generate_button = widgets.Button(
        description='Generate Recipe',
        disabled=False,
        button_style='success',
        tooltip='Click to generate recipe',
        icon='check'
    )
    
    output = widgets.Output()
    
    # Define button click behavior
    def on_button_clicked(b):
        with output:
            clear_output()
            
            # Get input values
            ingredients_list = [i.strip() for i in ingredients_input.value.split(',') if i.strip()]
            if not ingredients_list:
                print("⚠️ Please enter at least one ingredient")
                return
                
            servings = servings_slider.value
            
            dietary_prefs = {
                "vegetarian": vegetarian_checkbox.value,
                "vegan": vegan_checkbox.value,
                "glutenFree": gluten_free_checkbox.value,
                "lowCarb": low_carb_checkbox.value,
                "lowSodium": low_sodium_checkbox.value
            }
            
            # Generate recipe
            print(f"Generating recipe with: {', '.join(ingredients_list)}")
            print(f"Servings: {servings}")
            print(f"Dietary preferences: {', '.join([k for k, v in dietary_prefs.items() if v])}")
            
            recipe_data = get_recipe_with_nutrition(ingredients_list, servings, dietary_prefs)
            
            # Save JSON to file if requested
            if save_json_checkbox.value and recipe_data:
                filename = f"recipe_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                with open(filename, 'w') as f:
                    json.dump(recipe_data, f, indent=2)
                print(f"\n✓ Recipe saved to {filename}")
    
    generate_button.on_click(on_button_clicked)
    
    # Layout the widgets
    preferences_box = widgets.HBox([
        vegetarian_checkbox, vegan_checkbox, gluten_free_checkbox, 
        low_carb_checkbox, low_sodium_checkbox, save_json_checkbox
    ])
    
    app = widgets.VBox([
        widgets.HTML("<h2>Recipe Generator</h2>"),
        widgets.HTML("<p>Enter your ingredients and preferences below to generate a recipe</p>"),
        ingredients_input,
        servings_slider,
        widgets.HTML("<p><b>Dietary Preferences:</b></p>"),
        preferences_box,
        generate_button,
        output
    ])
    
    display(app)

In [12]:
#'Run the App (Execution Cell)'

# Run the interactive app
if __name__ == "__main__" or 'ipykernel' in sys.modules:
    create_interactive_recipe_app()